In [2]:
import numpy as np
from collections import namedtuple, deque
from heapq import heappush, heappop
from random import choice
from tqdm import tqdm
from icecream import ic

In [3]:
# Puzzle size
PUZZLE_DIM = 4
RANDOMIZE_STEPS = 10_000

# Action to represent the swap between two positions
action = namedtuple('Action', ['pos1', 'pos2'])

In [4]:
# Manhattan distance function (heuristic)
def manhattan_distance(state, goal_state):
    distance = 0
    goal_state_flat = np.array(goal_state).flatten()
    goal_state_flat = goal_state_flat.tolist()
    for i in range(len(state)):
        for j in range(len(state[i])):
            if state[i][j] != 0:  # Do not consider the empty space
                goal_pos = divmod(goal_state_flat.index(state[i][j]), len(state))
                distance += abs(i - goal_pos[0]) + abs(j - goal_pos[1])
    return distance

# Find the available actions (possible moves)
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions

# Perform an action on the state
def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

## A*

In [59]:
# Perform A* Search
def A_star(start_state, goal_state):
    # Initialize the frontier
    frontier = []

    # Initial cost and heuristic estimate
    g = 0  
    h = manhattan_distance(start_state, goal_state) 
    f = g + h  # Total estimated cost

    # Push the initial state into the frontier
    heappush(frontier, (f, start_state.tobytes(), g, None))

    # Dictionaries to store predecessors and costs
    came_from = {start_state.tobytes(): None}  # Maps a state to its predecessor (for reconstructing the path)
    g_score = {start_state.tobytes(): 0}  # Cost to reach each state from the start

    # Set to track explored states to prevent revisits
    explored = set()

    while frontier:
        # Extract the state with the smallest f-score
        _, current_bytes, current_g, parent = heappop(frontier)
        current_state = np.frombuffer(current_bytes, dtype=start_state.dtype).reshape(start_state.shape)

        # If the state has already been explored, skip it
        if current_bytes in explored:
            continue
        explored.add(current_bytes) 
        came_from[current_bytes] = parent  # Record the parent state

        # Check if the current state is the goal state
        if np.array_equal(current_state, goal_state):
            cost = len(explored)  # The total number of explored states
            return reconstruct_path(came_from, current_bytes), cost

        # Explore the neighbors of the current state
        for act in available_actions(current_state):
            neighbor = do_action(current_state, act)  # Apply action to get the neighbor state
            neighbor_bytes = neighbor.tobytes()

            # Calculate the tentative g-score (path cost through the current state)
            tentative_g = current_g + 1 
            if neighbor_bytes not in g_score or tentative_g < g_score[neighbor_bytes]:
                # If this path to the neighbor is shorter or unvisited, record it
                g_score[neighbor_bytes] = tentative_g
                f_score = tentative_g + manhattan_distance(neighbor, goal_state)
                heappush(frontier, (f_score, neighbor_bytes, tentative_g, current_bytes))

    return None, len(explored) 


# Reconstructs the path from the start state to the goal state using the predecessor dictionary
def reconstruct_path(came_from, current):
    path = []
    while current is not None:
        path.append(current)  # Add the current state to the path
        current = came_from[current]  # Move to the predecessor
    return path[::-1]  # Return the path in the correct order, from start to goal

In [ ]:
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))
    
start_state = state
goal_state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))

# Risolvi il puzzle
print("Initial state:")
print(state)


path, cost = A_star(start_state, goal_state)
quality = len(path) - 1  # Quality: Number of moves
print(f"Quality (number of moves): {quality}")
print(f"Cost (explored states): {cost}")
print("Solution Path:")
for step in path:
    print(np.frombuffer(step, dtype=start_state.dtype).reshape(start_state.shape))

Randomizing:   0%|          | 0/10000 [00:00<?, ?it/s]

Randomizing: 100%|██████████| 10000/10000 [00:00<00:00, 76335.85it/s]


Stato iniziale:
[[14  3  1  8]
 [ 7  2  5 13]
 [15 11  0 12]
 [10  6  4  9]]
Quality (number of moves): 50
Cost (explored states): 839064
Solution Path:
[[14  3  1  8]
 [ 7  2  5 13]
 [15 11  0 12]
 [10  6  4  9]]
[[14  3  1  8]
 [ 7  2  5 13]
 [15  0 11 12]
 [10  6  4  9]]
[[14  3  1  8]
 [ 7  0  5 13]
 [15  2 11 12]
 [10  6  4  9]]
[[14  3  1  8]
 [ 7  5  0 13]
 [15  2 11 12]
 [10  6  4  9]]
[[14  3  1  8]
 [ 7  5 11 13]
 [15  2  0 12]
 [10  6  4  9]]
[[14  3  1  8]
 [ 7  5 11 13]
 [15  2  4 12]
 [10  6  0  9]]
[[14  3  1  8]
 [ 7  5 11 13]
 [15  2  4 12]
 [10  6  9  0]]
[[14  3  1  8]
 [ 7  5 11 13]
 [15  2  4  0]
 [10  6  9 12]]
[[14  3  1  8]
 [ 7  5 11  0]
 [15  2  4 13]
 [10  6  9 12]]
[[14  3  1  8]
 [ 7  5  0 11]
 [15  2  4 13]
 [10  6  9 12]]
[[14  3  1  8]
 [ 7  5  4 11]
 [15  2  0 13]
 [10  6  9 12]]
[[14  3  1  8]
 [ 7  5  4 11]
 [15  0  2 13]
 [10  6  9 12]]
[[14  3  1  8]
 [ 7  0  4 11]
 [15  5  2 13]
 [10  6  9 12]]
[[14  0  1  8]
 [ 7  3  4 11]
 [15  5  2 13]
 [10  6  

## Bidirectional A*

In [5]:
# Reconstruct the full path from start to goal via the meeting point
def reconstruct_path_(came_from_start, came_from_goal, meeting_point):
    # Path from the start state to the meeting point
    path_start = []
    current = meeting_point
    while current is not None:
        path_start.append(current)
        current = came_from_start[current]
    path_start = path_start[::-1]  # Reverse to get the path in the correct order

    # Path from the meeting point to the goal state
    path_goal = []
    current = came_from_goal[meeting_point]
    while current is not None:
        path_goal.append(current)
        current = came_from_goal[current]

    return path_start + path_goal  # Combine the two paths

# Perform Bidirectional A* Search
def bidirectional_A_star(start_state, goal_state):
    # Initialize the frontiers for both directions
    frontier_start = []
    frontier_goal = []

    # Initial costs and heuristic estimates for both directions
    g_start, g_goal = 0, 0
    h_start = manhattan_distance(start_state, goal_state)
    h_goal = manhattan_distance(goal_state, start_state)
    f_start = g_start + h_start
    f_goal = g_goal + h_goal

    # Push the initial states into their respective frontiers
    heappush(frontier_start, (f_start, start_state.tobytes(), g_start, None))
    heappush(frontier_goal, (f_goal, goal_state.tobytes(), g_goal, None))

    # Dictionaries to store paths and costs for both directions
    came_from_start = {start_state.tobytes(): None}
    g_score_start = {start_state.tobytes(): 0}

    came_from_goal = {goal_state.tobytes(): None}
    g_score_goal = {goal_state.tobytes(): 0}

    # Sets to keep track of explored states
    explored_start = set()
    explored_goal = set()

# Bidirectional search loop
    while frontier_start and frontier_goal:
        # Expand from the start frontier
        if frontier_start:
            _, current_bytes, current_g, parent = heappop(frontier_start)
            current_state = np.frombuffer(current_bytes, dtype=start_state.dtype).reshape(start_state.shape)

            # Skip if already explored
            if current_bytes in explored_start:
                continue
            explored_start.add(current_bytes)
            came_from_start[current_bytes] = parent

            # Check if the reverse search has reached this state
            if current_bytes in came_from_goal:
                cost = len(explored_start) + len(explored_goal)
                return reconstruct_path_(came_from_start, came_from_goal, current_bytes), cost

            # Expand neighbors
            for act in available_actions(current_state):
                neighbor = do_action(current_state, act)
                neighbor_bytes = neighbor.tobytes()

                # Calculate tentive g-score
                tentative_g = current_g + 1
                if neighbor_bytes not in g_score_start or tentative_g < g_score_start[neighbor_bytes]:
                    g_score_start[neighbor_bytes] = tentative_g
                    f_score = tentative_g + manhattan_distance(neighbor, goal_state)
                    heappush(frontier_start, (f_score, neighbor_bytes, tentative_g, current_bytes))

        # Expand from the goal frontier
        if frontier_goal:
            _, current_bytes, current_g, parent = heappop(frontier_goal)
            current_state = np.frombuffer(current_bytes, dtype=goal_state.dtype).reshape(goal_state.shape)

            # Skip if already explored
            if current_bytes in explored_goal:
                continue
            explored_goal.add(current_bytes)
            came_from_goal[current_bytes] = parent

            # Check if the forward search has reached this state
            if current_bytes in came_from_start:
                cost = len(explored_start) + len(explored_goal)
                return reconstruct_path_(came_from_start, came_from_goal, current_bytes), cost

            # Expand neighbors
            for act in available_actions(current_state):
                neighbor = do_action(current_state, act)
                neighbor_bytes = neighbor.tobytes()

                # Calculate tentive g-score
                tentative_g = current_g + 1
                if neighbor_bytes not in g_score_goal or tentative_g < g_score_goal[neighbor_bytes]:
                    g_score_goal[neighbor_bytes] = tentative_g
                    f_score = tentative_g + manhattan_distance(neighbor, start_state)
                    heappush(frontier_goal, (f_score, neighbor_bytes, tentative_g, current_bytes))


    return None, len(explored_start) + len(explored_goal)


In [ ]:
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))
    
start_state = state
goal_state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))

# Risolvi il puzzle
print("Initial state:")
print(state)


path, cost = bidirectional_A_star(start_state, goal_state)
quality = len(path) - 1  # Quality: Number of moves
print(f"Quality (number of moves): {quality}")
print(f"Cost (explored states): {cost}")
print("Solution Path:")
for step in path:
    print(np.frombuffer(step, dtype=start_state.dtype).reshape(start_state.shape))

# 8.6s

Randomizing: 100%|██████████| 10000/10000 [00:00<00:00, 62464.32it/s]


Stato iniziale:
[[ 1 11 12  6]
 [10 14  2  3]
 [ 0 15  5  7]
 [13  4  9  8]]
Quality (number of moves): 48
Cost (explored states): 122824
Solution Path:
[[ 1 11 12  6]
 [10 14  2  3]
 [ 0 15  5  7]
 [13  4  9  8]]
[[ 1 11 12  6]
 [10 14  2  3]
 [15  0  5  7]
 [13  4  9  8]]
[[ 1 11 12  6]
 [10 14  2  3]
 [15  4  5  7]
 [13  0  9  8]]
[[ 1 11 12  6]
 [10 14  2  3]
 [15  4  5  7]
 [13  9  0  8]]
[[ 1 11 12  6]
 [10 14  2  3]
 [15  4  0  7]
 [13  9  5  8]]
[[ 1 11 12  6]
 [10 14  2  3]
 [15  0  4  7]
 [13  9  5  8]]
[[ 1 11 12  6]
 [10 14  2  3]
 [ 0 15  4  7]
 [13  9  5  8]]
[[ 1 11 12  6]
 [10 14  2  3]
 [13 15  4  7]
 [ 0  9  5  8]]
[[ 1 11 12  6]
 [10 14  2  3]
 [13 15  4  7]
 [ 9  0  5  8]]
[[ 1 11 12  6]
 [10 14  2  3]
 [13  0  4  7]
 [ 9 15  5  8]]
[[ 1 11 12  6]
 [10  0  2  3]
 [13 14  4  7]
 [ 9 15  5  8]]
[[ 1 11 12  6]
 [10  2  0  3]
 [13 14  4  7]
 [ 9 15  5  8]]
[[ 1 11  0  6]
 [10  2 12  3]
 [13 14  4  7]
 [ 9 15  5  8]]
[[ 1 11  6  0]
 [10  2 12  3]
 [13 14  4  7]
 [ 9 15  